In [2]:
import math
import pandas as pd
import pandas_datareader as pdr
import datetime 
import matplotlib.pyplot as plt
import numpy as np
aapl = pdr.get_data_yahoo('AAPL', 
                          start=datetime.datetime(2006, 10, 1), 
                          end=datetime.datetime(2011, 12, 30))


In [7]:
short_window = 40
long_window = 100

initial_capital = 100000

signals = pd.DataFrame(index=aapl.index)
signals['signal'] = 0.0

signals['short_mov_avg'] = aapl['Adj Close'].rolling(window=short_window, min_periods=1, center=False).mean()
signals['long_mov_avg'] = aapl['Adj Close'].rolling(window=long_window, min_periods=1, center=False).mean()

# Create signals (Came from course - dont understand)
signals['signal'][short_window:] = np.where(signals['short_mov_avg'][short_window:] 
                                            > signals['long_mov_avg'][short_window:], 1.0, 0.0)   
# Generate trading orders
signals['positions'] = signals['signal'].diff()

# Print `signals`
#print(signals.head(50))






positions = pd.DataFrame(index=signals.index)

positions['AAPL'] = 100*signals['signal'] 


# Initialize the portfolio with value owned   
portfolio = positions.multiply(aapl['Adj Close'], axis=0)

# Store the difference in shares owned 
pos_diff = positions.diff()

# Add `holdings` to portfolio
portfolio['holdings'] = (positions.multiply(aapl['Adj Close'], axis=0)).sum(axis=1)

# Add `cash` to portfolio
portfolio['cash'] = initial_capital - (pos_diff.multiply(aapl['Adj Close'], axis=0)).sum(axis=1).cumsum()   

# Add `total` to portfolio
portfolio['total'] = portfolio['cash'] + portfolio['holdings']

# Add `returns` to portfolio
portfolio['returns'] = portfolio['total'].pct_change()

# Print the first lines of `portfolio`
print(portfolio)


                   AAPL     holdings          cash          total   returns
Date                                                                       
2006-10-02     0.000000     0.000000  100000.00000  100000.000000       NaN
2006-10-03     0.000000     0.000000  100000.00000  100000.000000  0.000000
2006-10-04     0.000000     0.000000  100000.00000  100000.000000  0.000000
2006-10-05     0.000000     0.000000  100000.00000  100000.000000  0.000000
2006-10-06     0.000000     0.000000  100000.00000  100000.000000  0.000000
...                 ...          ...           ...            ...       ...
2011-12-23  1231.622696  1231.622696   99452.45285  100684.075546  0.000145
2011-12-27     0.000000     0.000000  100693.84737  100693.847370  0.000097
2011-12-28     0.000000     0.000000  100693.84737  100693.847370  0.000000
2011-12-29     0.000000     0.000000  100693.84737  100693.847370  0.000000
2011-12-30     0.000000     0.000000  100693.84737  100693.847370  0.000000

[1323 rows 